In [177]:
#данный скрипт: 

## - осуществляет парсинг еженедельных чартов
### - через requests: Spotify
### - через selenium: Youtube

## - должен запускаться каждую неделю один раз в неделю
### - и Youtube, и spotify начинают неделю в пятницу и заканчивают в четверг
### - поэтому запускать данный скрипт надо ранним утром в пятницу

## - на выходе:
### - обновляет уже хранящиеся данные в csv файлах каждого стриминга, лежащие в корневой директории

In [234]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import datetime
from datetime import datetime, date, time, timezone
from dateutil.relativedelta import relativedelta

In [179]:
#установка и импорт selenium
!pip install selenium
from selenium import webdriver as wb
!pip install chromedriver

In [211]:
#задаем команду для получения даты
currentDT = datetime.datetime.now() 

In [229]:
# технический код, созданный для момента, когда не было исходных файлов в директории
#for i in ["spotify", "youtube"]:
#    cols = ['rank', 'title', 'artist', "date", "streams"]
#    name = "all_" + i + ".csv" 
#    pd.DataFrame(columns = cols).to_csv(name, encoding= "utf-8")

### Youtube Top 100 Tracks Russia

In [230]:
#selenium-код
url='https://charts.youtube.com/charts/TopSongs/ru?hl=ru'
br = wb.Chrome("/Users/sergey/chromedriver") #заменить на директорию, в которой лежит нужный драйвер 
br.get(url)
sleep(randint(3,4))
generated_html = br.page_source
br.quit()

In [235]:
#работаем с html
soup = BeautifulSoup(generated_html, 'html.parser')

all_together = soup.findAll('span', attrs={'class':'ytmc-ellipsis-text style-scope'})
songs = video_and_artists[2:][0::2]
artists = video_and_artists[2:][1::2]

songs_clean = [i.get_text() for i in songs]
artists_clean = [i.get_text() for i in artists]


views = soup.findAll('span', attrs={'class':'style-scope ytmc-chart-table'})[4::5]
views_clean = [i.get_text() for i in views]

youtube_tracks_top_100 = pd.DataFrame()
youtube_tracks_top_100['title'] = songs_clean
youtube_tracks_top_100['artist'] = artists_clean
youtube_tracks_top_100['streams'] = views_clean
youtube_tracks_top_100['rank'] = youtube_tracks_top_100.reset_index().index +1
youtube_tracks_top_100= youtube_tracks_top_100[['rank', 'title', 'artist', 'streams']]
youtube_tracks_top_100["date"] = currentDT.strftime("%d/%m/%Y")  

#записываем неделю 
date_start = currentDT - relativedelta(days=+7)
date_end = currentDT - relativedelta(days=+1)
week = datetime.strftime(date_start,"%d/%m/%y") + " - " + datetime.strftime(date_end,"%d/%m/%y")
youtube_tracks_top_100["week"] = week


In [240]:
#сохраняем html для использования на сайте (через Make_weekly_charts.py)
youtube_tracks_top_100.to_html("current_youtube_html.html", encoding = "utf-8")

In [236]:
#берем имеющийся в корневой директории csv файл и обновляем его

all_youtube = pd.read_csv("all_youtube.csv")
all_youtube = all_youtube.drop(all_youtube.columns[[0]], axis=1) #удаляем получающуюся после импорта лишнюю колонку 
frames = [all_youtube, youtube_tracks_top_100]
all_youtube = pd.concat(frames, sort=False)
all_youtube.to_csv("all_youtube.csv", encoding = "utf-8")

### Spotify

In [237]:
#базовая ссылка на последний актуальный еженедельный чарт по России
base_url = 'https://spotifycharts.com/regional/ru/weekly/latest'
r = requests.get(base_url)
#на всякий случай поставим на паузу
sleep(randint(1,3))
soup = BeautifulSoup(r.text, 'html.parser')
chart = soup.find('table', {'class': 'chart-table'})
tbody = chart.find('tbody')
all_rows = []


#сам скрэйпинг
for tr in tbody.find_all('tr'):
    #позиция трека
    rank_text = tr.find('td', {'class': 'chart-table-position'}).text
    #артист
    artist_text = tr.find('td', {'class': 'chart-table-track'}).find('span').text
    artist_text = artist_text.replace('by ','').strip()
    #название трека
    title_text = tr.find('td', {'class': 'chart-table-track'}).find('strong').text
    #кол-во стримов для трека
    streams_text = tr.find('td', {'class': 'chart-table-streams'}).text
    #cборка таблицы (цикл на случай парсинга нескольких чартов)
    all_rows.append( [rank_text, title_text, artist_text, streams_text] )
    
#создаем читаемый датафрейм в pandas
rus_spotify_top_200 = pd.DataFrame(all_rows, columns =['rank','title', "artist",'streams'])
rus_spotify_top_200["date"] = currentDT.strftime("%d/%m/%Y")  

#записываем неделю 
date_start = currentDT - relativedelta(days=+7)
date_end = currentDT - relativedelta(days=+1)
week = datetime.strftime(date_start,"%d/%m/%y") + " - " + datetime.strftime(date_end,"%d/%m/%y")
rus_spotify_top_200["week"] = week



In [238]:
#сохраняем html spotify для использования на сайте (через Make_weekly_charts.py)
rus_spotify_top_200.to_html("current_spotify_html.html", encoding = "utf-8")

In [239]:
#берем имеющийся в корневой директории csv файл и обновляем его

all_spotify = pd.read_csv("all_spotify.csv")
all_spotify = all_spotify.drop(all_spotify.columns[[0]], axis=1) #удаляем получающуюся после импорта лишнюю колонку 
frames = [all_spotify, rus_spotify_top_200]
all_spotify = pd.concat(frames, sort=False)
all_spotify.to_csv("all_spotify.csv", encoding = "utf-8")